In [ ]:
import os
import glob as gb
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from skimage.io import imread, imshow
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Add, Dropout,Dense, Activation, ZeroPadding2D,\
               BatchNormalization, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint


## Loading Training dataset

In [ ]:
#get the data path
trainpath= ('................/')

# get the training classes
keys=os.listdir(trainpath)
values=range(len(keys))

training_classes = dict (zip(keys,values))
print('training_classes = ',training_classes)

In [ ]:
#Loading and resizing training data
new_size=224    
X_train = []
y_train = []
train_label=[]
folder_item_numbers = []

for folder in  os.listdir(trainpath ) : 
    train_label.append(folder)
    files = gb.glob(pathname= str( trainpath  + folder + '/*.jpg'))
    folder_item_numbers.append(len(files))
    for file in files: 
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        X_train.append(resized_image)
        y_train.append(training_classes[folder])

print('--------------------------------------------------')        
foldernames=pd.DataFrame({'Folder_name':train_label})
itemnumbers=pd.DataFrame({'Traning Image Numbers':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('--------------------------------------------------')        
#check items in X_test
print("items in X_train is:       ",len(X_train) , " items") 
print("items in y_train is:       ",len(y_train) , " items") 

In [ ]:
#showing training images with labels
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_train),12))) : 
    plt.subplot(3,4,n+1)
    plt.imshow(X_train[i])   
    plt.axis('off')
    def ImageClass(n):
        for x , y in training_classes.items():
            if n == y :
                return x
    plt.title(ImageClass(y_train[i]))

In [ ]:
#shuffle data
img_size=224
X_train,y_train = shuffle(X_train, y_train)
# Split the training datdset into two groups
x_train, x_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size= 0.8)
print('Training Samples =:       ',len(x_train) ,'  and the number of labels is     ', len(y_train))
print('Validation Samples =:       ',len(x_valid) ,'  and the number of labels is     ', len(y_valid))


In [ ]:
#converting all TRAIN data to array
x_train = np.array(x_train) 
y_train = np.array(y_train)

x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

y_train = to_categorical(y_train,len(train_label))
y_valid = to_categorical(y_valid,len(train_label))

print("x_train shape  :" ,x_train.shape)
print("y_train shape :", y_train.shape)
print('-----------------------------------------')
print("x_valid shape  :" ,x_valid.shape)
print("y_valid shape :", y_valid.shape)


In [ ]:
#shuffle data
x_train,y_train = shuffle(x_train,y_train)
x_valid,y_valid = shuffle(x_valid,y_valid)
#normalizing data
x_train=x_train/255.0
x_valid =x_valid/255.0

## building the model

In [ ]:
# loading ResNet50
# Loading the pretrained model without the output of the last convolution block 
base_model = ResNet50(include_top=False, input_shape=(224, 224, 3), weights = 'imagenet')

# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)
# Add a fully connected layer with 2048 hidden units and ReLU activation
x = Dense(2048, activation='relu')(x)
# Adding a fully connected layer having len(image_class) neurons which will  give the probability of image 
predictions = Dense(len(train_label), activation='softmax')(x)

base_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# let's visualize layer names and layer indices to see the layers we will freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

In [ ]:
base_model.summary()
        

In [ ]:
#define the fine-tuning layers
train_layers=[176,171,168,165]

for i in train_layers:
    print('Layer name is:      ',base_model.layers[i].name)
    print('the input:   ',base_model.layers[i].input)
    print('the output:  ',base_model.layers[i].output)
    print('....................................................................')

In [ ]:
def  training_opt(filename):
    
    file_dir='...............'  # save path
    filepath = os.path.join(file_dir,filename+'.hdf5')     # Save the model with the same layer name
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True, verbose=1)   
    earlystop=EarlyStopping( monitor="val_loss",  min_delta=0,   patience=15,  mode="auto")
    model_callbacks = [checkpoint, earlystop]
    return model_callbacks


In [ ]:
optimize=Adam(learning_rate=0.001)
epochs=100
batch_size=64

for i in train_layers:
    Layername=base_model.layers[i].name
    print('fine_tuning layer name: -----------------------------    ', Layername)
    
    for layer in base_model.layers[i:-1]:
        layer.trainable = True
    
    ResNet50_model = Model(inputs=base_model.input, outputs=predictions)
    ResNet50_model.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['accuracy']) 
    history=ResNet50_model.fit(x_train,y_train, validation_data= (x_valid,y_valid), epochs=epochs, batch_size=batch_size, callbacks=training_opt(Layername), verbose=1)
    

In [ ]:
#showing results and model accuracy 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(12, 6))
plt.grid()

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Loading the Test dataset

In [ ]:
# get the path of the data set
testpath = ('......................./')

keys=os.listdir(testpath)
values=range(len(keys))

Test_classes = dict (zip(keys,values))
print('Test_classes = ',Test_classes)

In [ ]:
#Loading and resizing of test data
new_size=224    
x_test = []
y_test = []
test_labels=[]
folder_item_numbers = []

for folder in  os.listdir(testpath ) : 
    test_labels.append(folder)
    files = gb.glob(pathname= str( testpath  + folder + '/*.jpg'))
    folder_item_numbers.append(len(files))
    for file in files: 
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        x_test.append(resized_image)
        y_test.append(Test_classes[folder])
        
print('--------------------------------------------------')        
foldernames=pd.DataFrame({'Label_name':test_labels})
itemnumbers=pd.DataFrame({'Number of samples':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('--------------------------------------------------')        
#check items in X_test
print("items in x_test is:       ",len(x_test) , " items") 
print("items in y_test is:       ",len(y_test) , " items") 

In [ ]:
#converting all test data to array
x_test = np.array(x_test)
y_test = np.array(y_test)
y_test = to_categorical(y_test,len(test_labels))

print('-----------------------------------------')
print("x_test shape  :" ,x_test.shape)
print("y_test shape :", y_test.shape)
#shuffle data
x_test,y_test = shuffle(x_test,y_test)
#normalizing data
x_test =x_test/255.0

In [ ]:
from tensorflow.keras.models import load_model
# loading the fine-tuning model
model=load_model('..................../conv5_block3_1_conv.hdf5') 

In [ ]:
y_test_pred = model.predict(x_test)
# predicted probabilities generated by the model
y_prediction = np.argmax(y_test_pred, axis=1)
# ground truth labels
y_true = np.argmax(y_test, axis=1)

y_true.shape, y_prediction.shape

In [ ]:
y_prediction

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_true, y_prediction)
cm



In [ ]:
from mlxtend.plotting import plot_confusion_matrix

plt.rcParams.update({'font.size': 16})

fig, ax = plot_confusion_matrix(conf_mat=cm,  figsize=(8, 8),
                                colorbar=False,
                                show_absolute=True,
                                show_normed=False,
                                class_names=test_labels,cmap="OrRd")

plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_prediction, target_names=test_labels))

In [ ]:
# compute sensitivity and specificity for multi classification 

from sklearn.metrics import precision_recall_fscore_support
res = []
for l in list(range(len(test_labels))):
    prec,recall,_,_ = precision_recall_fscore_support(np.array(y_true)==l,
                                                      np.array(y_prediction)==l,
                                                      pos_label=True,average=None)
    res.append([l,recall[0],recall[1]])
    
pd.DataFrame(res,columns = ['class','sensitivity','specificity'])

In [ ]:
#overall sensitivity and specificity
df[1:].mean(axis = 0)

In [ ]:
y_test.shape, y_test_pred.shape

In [ ]:
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle


fpr = dict()
tpr = dict()
roc_auc = dict()
lw=2
n_classes=len(test_labels)

plt.figure(figsize=(9,9))
plt.grid()
plt.rcParams['font.size']=14

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_test_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['blue', 'red', 'green','orange'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='AUC for class {0} ={1:0.2f}' ''.format(test_labels[i], roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize = 16)
plt.ylabel('True Positive Rate', fontsize = 16)
#plt.title('ROC for brain tumor dataset', fontsize = 16)
plt.legend(loc="lower right", fontsize = 14)
plt.show()